# Day 3: Gear Ratios
https://adventofcode.com/2023/day/3
## Part 1
The elf and I have arrived at the gondola station, but the gondolas are not working because a part is missing.

To solve this puzzle, the "part numbers", which are the numbers adjacent to symbols in the "schematic", need to be summed. A symbol is anything other than a digit or a period. In the example below, all numbers except 114 and 58 are part numbers. The sum of all these part numbers is 4361.

Strategy: In each line, begin at the leftmost position, p, and read a character at a time in the current line, cl. If the character is a digit, note the position of the beginning digit, b. Continue reading until you reach a period, a symbol, or the end of the line. Note the position, ep1, one position after the ending digit. Determine whether there are symbols adjacent to the number: consider the positions to the left and right, and the positions in the line above, la, and the line below, lb, including adjacent corners. Be aware of the edges of the schematic. If a symbol is found, calculate the value of the entire decimal number and add it to a running total.

In [1]:
sample_input = '467..114..\n...*......\n..35..633.\n......#...\n617*......\n.....+.58.\n..592.....\n......755.\n...$.*....\n.664.598..'
schematic = sample_input.split('\n')

cl = 2
schematic[cl]

'..35..633.'

In [2]:
def find_first_digit(s):
    p = 0
    while p < len(s):
        if s[p].isnumeric():
            return p
        p = p + 1
    return p

find_first_digit('..345..')

2

In [3]:
def find_first_nondigit(s):
    p = 0
    while p < len(s):
        if not s[p].isnumeric():
            return p
        p = p + 1
    return p

find_first_nondigit('587*..')

3

Find the first number in a line

In [4]:
s = schematic[0]
b = find_first_digit(s)
e = find_first_nondigit(s[b:])

[b, e, s[b:b+e]]

[0, 3, '467']

Find the first number in the remaining substring

In [5]:
s = s[b+e:]
b = find_first_digit(s)
e = find_first_nondigit(s[b:])

[b, e, s[b:b+e]]

[2, 3, '114']

Identify symbols

In [6]:
def contains_symbol(s):
    for p in range(len(s)):
        if not s[p].isnumeric():
            if not s[p] == '.':
                return True
    return False

contains_symbol('..234..&')

True

Get the characters above or below a number, extending to the left and right by one position, unless you're at the edge.

In [7]:
def get_la(schematic, cl, b, ep1):
    if cl <= 0:
        return ''
    else:
        if b > 0: 
            b = b-1
        if ep1 < len(schematic[cl]):
            ep1 = ep1 + 1
        return schematic[cl-1][b:ep1]

get_la(sample_input.split('\n'), 9, 1, 4)

'...$.'

In [8]:
def get_lb(schematic, cl, b, ep1):
    if cl+1 >= len(schematic):
        return ''
    else:
        if b > 0: 
            b = b-1
        if ep1 < len(schematic[cl]):
            ep1 = ep1 + 1
        return schematic[cl+1][b:ep1]

get_lb(sample_input.split('\n'), 0,0,3)

'...*'

Identify all the numbers with their locations and whether there's an adjacent symbol. Keep a running sum of the part numbers.

In [9]:
def solve_part1(schematic, verbose=False):
    sum=0
    for cl in range(len(schematic)):
        s = schematic[cl]
        p = 0
        b = 0
        e = 0
        while p < len(s):
            b = find_first_digit(s[p:])        
            if b < len(s[p:]):
                e = find_first_nondigit(s[p+b:])
                n = int(s[p+b:p+b+e])
                pn = False
                if p+b > 0:
                    pn = (contains_symbol(s[p+b-1]))
                if p+b+e < len(s):
                    pn = (pn or contains_symbol(s[p+b+e]))
                pn = (pn or contains_symbol(get_la(schematic, cl, p+b, p+b+e)))
                pn = (pn or contains_symbol(get_lb(schematic, cl, p+b, p+b+e)))
                if pn:
                    sum = sum + n
                if verbose: print(f'[{cl}][{p+b}:{p+b+e}] {n}  {pn}')            
            p = p+b+e
            b = 0
            e = 0
    return sum
    
solve_part1(sample_input.split('\n'), verbose=True)

[0][0:3] 467  True
[0][5:8] 114  False
[2][2:4] 35  True
[2][6:9] 633  True
[4][0:3] 617  True
[5][7:9] 58  False
[6][2:5] 592  True
[7][6:9] 755  True
[9][1:4] 664  True
[9][5:8] 598  True


4361

In [10]:
with open('data/day03_in.txt', 'r') as f:
    puzzle_input = f.read()

In [11]:
solve_part1(puzzle_input.split('\n'), verbose=False)

536576

536576 is correct for my puzzle input.

# Part 2

Now identify gears: * symbols that are adjacent to exactly two part numbers. The gear ratio of the gear is the product of those numbers. Add the sum of all gear ratios.

For the sample input, the solution is 467 * 35 + 755 * 598 = 467835

Strategy: first, find a * symbol. Look for digits to the left and right, above and below, and in the corners, being aware of edges. Count the number of adjacent numbers, keeping in mind that consecutive digits in upper and lower rows are part of the same number. If there are two adjacent numbers, extend left and right to get both entire numbers. Multiply them together and keep a running total.

Or, we could build a dictionary of part numbers that are adjacent to * symbols by slightly modifying the part 1 code.

In [12]:
def contains_gear_symbol(s):
    for p in range(len(s)):
        if not s[p].isnumeric():
            if s[p] == '*':
                return True
    return False

contains_gear_symbol('.*234..&')

True

In [38]:
def find_numbers(schematic):
    '''Returns a dictionary indexed by row number, where each entry is a list of lists describing any numbers in that row that are adjacent to stars.
    Each of the inner lists contains the position of the first digit, the next position after the last digit, and the value of the number.
    '''
    numbers = {}
    for cl in range(len(schematic)):
        s = schematic[cl]
        numbers[cl] = []
        p = 0
        b = 0
        e = 0
        while p < len(s):
            b = find_first_digit(s[p:])        
            if b < len(s[p:]):
                e = find_first_nondigit(s[p+b:])
                n = int(s[p+b:p+b+e])
                pn = False
                if p+b > 0:
                    pn = (contains_gear_symbol(s[p+b-1]))
                if p+b+e < len(s):
                    pn = (pn or contains_gear_symbol(s[p+b+e]))
                pn = (pn or contains_gear_symbol(get_la(schematic, cl, p+b, p+b+e)))
                pn = (pn or contains_gear_symbol(get_lb(schematic, cl, p+b, p+b+e)))
                if pn:
                    numbers[cl].append([p+b, p+b+e, n])
            p = p+b+e
            b = 0
            e = 0
    return numbers

find_numbers(sample_input.split('\n'))

{0: [[0, 3, 467]],
 1: [],
 2: [[2, 4, 35]],
 3: [],
 4: [[0, 3, 617]],
 5: [],
 6: [],
 7: [[6, 9, 755]],
 8: [],
 9: [[5, 8, 598]]}

In [39]:
def find_stars(row):
    found = []
    start = 0
    while start < len(row):
        look = row.find('*', start)
        if look >=0:
            found.append(look)
            start = look+1
        else:
            start = len(row)
    return(found)
        
find_stars(sample_input.split()[1])

[3]

So for each row of the schematic, find all the stars. Get star-adjacent part numbers in the current row plus the row above and the row below. Identify how many part numbers are adjacent to each particular star. If there are exactly two numbers, multiply them and add them to the running total.

In [46]:
def touching(number, star):
    first = number[0]
    last = number[1]
    touching = first <= star+1 and last >= star
    return touching
    
touching([4, 6, 99], 7)  # the number 99 is in positions 4 and 5

False

In [48]:
def adjacent_only(numbers, star):
    adjacent = []
    for number in numbers:
        if touching(number, star):
            adjacent.append(number)
    return adjacent

adjacent_only([[1,3,99], [7,9,36]], 6)

[[7, 9, 36]]

In [67]:
def solve_part2(schematic, verbose=False):
    sum=0
    numbers = find_numbers(schematic)
    row = 0
    while row in range(len(schematic)):
        stars = find_stars(schematic[row])
        for star in stars:
            adjacent = []
            if verbose: print(f'\nStar position in row {row}: {star}')
            if verbose: print(f'Adjacent numbers in this row: {adjacent_only(numbers[row], star)}')
            adjacent.extend(adjacent_only(numbers[row], star))
                        
            if row > 0:
                if verbose: print(f'Adjacent numbers in upper row: {adjacent_only(numbers[row-1], star)}')
                adjacent.extend(adjacent_only(numbers[row-1], star))
                
            if row < len(schematic) - 1:
                if verbose: print(f'Adjacent numbers in lower row: {adjacent_only(numbers[row+1], star)}')
                adjacent.extend(adjacent_only(numbers[row+1], star))

            if len(adjacent) == 2:
                gear_ratio = adjacent[0][2] * adjacent[1][2]
                if verbose: print(f'Gear ratio: {gear_ratio}')
                sum = sum + gear_ratio
        row = row + 1
    return sum

solve_part2(sample_input.split('\n'), verbose=True)


Star position in row 1: 3
Adjacent numbers in this row: []
Adjacent numbers in upper row: [[0, 3, 467]]
Adjacent numbers in lower row: [[2, 4, 35]]
Gear ratio: 16345

Star position in row 4: 3
Adjacent numbers in this row: [[0, 3, 617]]
Adjacent numbers in upper row: []
Adjacent numbers in lower row: []

Star position in row 8: 5
Adjacent numbers in this row: []
Adjacent numbers in upper row: [[6, 9, 755]]
Adjacent numbers in lower row: [[5, 8, 598]]
Gear ratio: 451490


467835

In [68]:
solve_part2(puzzle_input.split('\n'), verbose=False)

75741499

75741499 is correct. 